# SQL Server / SQL Database Machine Learning Service
## Create **Motor Trend Car Road Tests** Table

In [20]:
DROP TABLE IF EXISTS [dbo].[MotorTrendCarRoadTests]
GO
CREATE TABLE [dbo].[MotorTrendCarRoadTests] (
    [mpg] FLOAT NOT NULL     -- Miles/(US) gallon
    , [cyl] INT NOT NULL     -- Number of cylinders
    , [hp] FLOAT NOT NULL    -- Gross horsepower
    , [disp] FLOAT NOT NULL  -- Displacement (cu.in.)
    , [drat] FLOAT NOT NULL  -- Rear axle ratio
    , [wt] FLOAT NOT NULL    -- Weight (1000 lbs)
    , [qsec] FLOAT NOT NULL  -- 1/4 mile time
    , [vs] INT NOT NULL      -- Engine (0 = V-shaped, 1 = straight)
    , [am] INT NOT NULL      -- Transmission (0 = automatic, 1 = manual)
    , [gear] INT NOT NULL    -- Number of forward gears
    , [carb] INT NOT NULL    -- Number of carburetors
);

## Import data from R dataset

In [21]:
INSERT INTO dbo.MotorTrendCarRoadTests
EXEC sp_execute_external_script
        @language = N'R'
        , @script = N'motor_trend_car_road_tests <- mtcars;'
        , @input_data_1 = N''
        , @output_data_1_name = N'motor_trend_car_road_tests'

## Train & Save Model
### Create the *Training* Stored Procedure 
Creating a stored procedure to train a Logistic Regression Model

In [22]:
DROP PROCEDURE IF EXISTS dbo.train_manual_transmission_model_v1;
GO
CREATE PROCEDURE dbo.train_manual_transmission_model_v1
(   
    @model_name nvarchar(30) OUTPUT
    , @model_version int OUTPUT
    , @trained_model varbinary(max) OUTPUT
)
AS
BEGIN
    EXEC sp_execute_external_script
    @language = N'R'
    , @script = N'
        lr_model <- glm(formula = am ~ hp + wt, data = MotorTrendCarRoadTests, family = binomial);

        trained_model <- as.raw(serialize(lr_model, connection=NULL));
        model_name <- "manual_transmission_model";
        model_version <- as.integer(1);
    '
    , @input_data_1 = N'SELECT [hp], [wt], [am] FROM MotorTrendCarRoadTests'
    , @input_data_1_name = N'MotorTrendCarRoadTests'
    , @params = N'@model_name nvarchar(30) OUTPUT, @model_version int OUTPUT, @trained_model varbinary(max) OUTPUT'
	, @model_name = @model_name OUTPUT
    , @model_version = @model_version OUTPUT
    , @trained_model = @trained_model OUTPUT;
END;

### Using RevoScaleR

In [23]:
DROP PROCEDURE IF EXISTS dbo.train_manual_transmission_model_v2;
GO
CREATE PROCEDURE dbo.train_manual_transmission_model_v2
(   
    @model_name nvarchar(30) OUTPUT
    , @model_version int OUTPUT
    , @trained_model varbinary(max) OUTPUT
)
AS
BEGIN
    EXEC sp_execute_external_script
    @language = N'R'
    , @script = N'
        require("RevoScaleR");

        lr_model <- rxLogit(formula = am ~ hp + wt, data = MotorTrendCarRoadTests);

        trained_model <- rxSerializeModel(lr_model, realtimeScoringOnly = TRUE);
        model_name <- "manual_transmission_model";
        model_version <- as.integer(2);
    '
    , @input_data_1 = N'SELECT [hp], [wt], [am] FROM MotorTrendCarRoadTests'
    , @input_data_1_name = N'MotorTrendCarRoadTests'
    , @params = N'@model_name nvarchar(30) OUTPUT, @model_version int OUTPUT, @trained_model varbinary(max) OUTPUT'
	, @model_name = @model_name OUTPUT
    , @model_version = @model_version OUTPUT
    , @trained_model = @trained_model OUTPUT;
END;

### Create a table for the machine learning models

In [24]:
DROP TABLE IF EXISTS [dbo].[MachineLearningModels]
GO
CREATE TABLE [dbo].[MachineLearningModels] (
    model_name nvarchar(30) not null default('default model')
    , model_version int not null default(1)
    , model varbinary(max) not null
    , created_on datetime not null default(getdate())
    , last_updated_on datetime not null default(getdate())
    CONSTRAINT PK_MachineLearningModels PRIMARY KEY (model_name, model_version)	
);

### Train & Save Regression Model

In [25]:
DECLARE @model_name NVARCHAR(30), @model_version INT, @model VARBINARY(MAX);

EXEC dbo.train_manual_transmission_model_v1 @model_name OUTPUT, @model_version OUTPUT, @model OUTPUT;

DELETE
FROM    [dbo].[MachineLearningModels] 
WHERE   model_name = @model_name
AND     model_version = @model_version;

INSERT
INTO    [dbo].[MachineLearningModels] 
        (model_name, model_version, model)
VALUES  (@model_name, @model_version, @model);

In [26]:
DECLARE @model_name NVARCHAR(30), @model_version INT, @model VARBINARY(MAX);

EXEC dbo.train_manual_transmission_model_v2 @model_name OUTPUT, @model_version OUTPUT, @model OUTPUT;

DELETE
FROM    [dbo].[MachineLearningModels] 
WHERE   model_name = @model_name
AND     model_version = @model_version;

INSERT
INTO    [dbo].[MachineLearningModels] 
        (model_name, model_version, model)
VALUES  (@model_name, @model_version, @model);

## Prediction
### Via Stored Procedure

In [27]:
DROP PROCEDURE IF EXISTS dbo.predict_manual_transmission;
GO
CREATE PROCEDURE dbo.predict_manual_transmission
(   
    @hp FLOAT
    , @wt FLOAT
    , @am FLOAT OUTPUT
)
AS
BEGIN
    DECLARE @model VARBINARY(MAX) = 
    (SELECT TOP(1) model 
    FROM [dbo].[MachineLearningModels] 
    WHERE model_name = 'manual_transmission_model' 
    AND model_version = 1);

    EXEC sp_execute_external_script
    @language = N'R'
    , @script = N'
        lr_model <- unserialize(as.raw(trained_model));
        
        newdata <- data.frame(hp = hp, wt = wt);
        am <- predict(lr_model, newdata, type = "response");
    '       
    , @params = N'@trained_model varbinary(max), @hp FLOAT, @wt FLOAT, @am FLOAT OUTPUT'
	, @trained_model = @model
    , @hp = @hp
    , @wt = @wt
    , @am = @am OUTPUT;
END

In [28]:
DECLARE @am FLOAT;
EXEC dbo.predict_manual_transmission 120, 2.8, @am OUTPUT;
SELECT @am AS ManualTransmissionPropability;

## Native Scoring

In [29]:
DECLARE @model VARBINARY(MAX) = (SELECT TOP(1) model FROM [dbo].[MachineLearningModels] WHERE model_name = 'manual_transmission_model' AND model_version = 2);
SELECT d.*, p.* 
FROM PREDICT(MODEL = @model, DATA = [dbo].[MotorTrendCarRoadTests] AS d) WITH (am_Pred float) AS p;